In [ ]:
import datasets as ds

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
import datasets as ds
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
stop_words = set(stopwords.words('english'))

# Load the legal documentation dataset
dataset = ds.load_dataset('lex_glue', 'ecthr_b')
features = dataset['train']['text']
labels = dataset['train']['labels']

# Text cleaning and tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text[2:])
    text= [w for w in tokenizer.tokenize(text) if not w in stop_words]
    return text

# Apply text cleaning to facts
features = [[preprocess_text(fact) for fact in text] for text in features]

# Create a DataFrame for easy handling
df = pd.DataFrame({'facts': features, 'labels': labels})

# Remove duplicates


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df

,facts,labels
0,"[[beginning, events, relevant, application, k,...",[4]
1,"[[applicant, monarch, liechtenstein, born, 194...","[8, 3, 9]"
2,"[[june, 1949, plots, agricultural, land, owned...",[3]
3,"[[1991, mr, du, ##an, sl, ##ob, ##od, ##nk, re...","[6, 8, 5]"
4,"[[applicant, italian, citizen, born, 1947, liv...","[8, 3]"
...,...,...
8995,"[[applicant, born, 1960, lives, village, kara,...","[3, 4]"
8996,"[[applicant, born, 1946, currently, detention,...",[]
8997,"[[applicants, russian, nationals, material, ti...","[0, 1, 2]"
8998,"[[cases, applicants, brought, civil, proceedin...","[3, 9]"


In [7]:

# Split into training and validation sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Define PyTorch Dataset
class LegalDataset(Dataset):
    def __init__(self, text_list, label_list, tokenizer):
        self.text_list = text_list
        self.label_list = label_list
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        label = self.label_list[idx]

        encoding = self.tokenizer(text, truncation=True, padding=True, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create PyTorch Datasets
train_dataset = LegalDataset(train_df['facts'].tolist(), train_df['labels'].tolist(), tokenizer)
valid_dataset = LegalDataset(valid_df['facts'].tolist(), valid_df['labels'].tolist(), tokenizer)

# Define DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# Fine-tune DistilBERT model with PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(df['labels'].iloc[0]))
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the fine-tuned model
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in tqdm(valid_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].numpy()

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

print("Fine-tuned Model Classification Report:")
print(classification_report(all_labels, all_preds))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3:   0%|          | 0/900 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [7938] at entry 0 and [1890] at entry 1